In [2]:
from sfm import SFM
import os, cv2
import numpy as np
import matplotlib.pyplot as plt

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [80]:
src = 'datasets/templeRing'
detector = cv2.BRISK_create()

In [81]:
files = os.listdir(src)
files.sort()
itoimg = {}
n = 0

for f in files:
    try:
        im = cv2.imread(f"{src}/{f}")
        im.shape

        itoimg[n] = f"{src}/{f}"
        n+=1

    except:
        pass

print(itoimg)

{0: 'datasets/templeRing/00.png', 1: 'datasets/templeRing/01.png', 2: 'datasets/templeRing/02.png', 3: 'datasets/templeRing/03.png', 4: 'datasets/templeRing/04.png', 5: 'datasets/templeRing/05.png', 6: 'datasets/templeRing/06.png', 7: 'datasets/templeRing/07.png', 8: 'datasets/templeRing/08.png', 9: 'datasets/templeRing/09.png', 10: 'datasets/templeRing/10.png', 11: 'datasets/templeRing/11.png', 12: 'datasets/templeRing/12.png', 13: 'datasets/templeRing/13.png', 14: 'datasets/templeRing/14.png', 15: 'datasets/templeRing/15.png', 16: 'datasets/templeRing/16.png', 17: 'datasets/templeRing/17.png', 18: 'datasets/templeRing/18.png', 19: 'datasets/templeRing/19.png', 20: 'datasets/templeRing/20.png', 21: 'datasets/templeRing/21.png', 22: 'datasets/templeRing/22.png', 23: 'datasets/templeRing/23.png', 24: 'datasets/templeRing/24.png', 25: 'datasets/templeRing/25.png', 26: 'datasets/templeRing/26.png', 27: 'datasets/templeRing/27.png', 28: 'datasets/templeRing/28.png', 29: 'datasets/templeRin

In [5]:
def threshold(img, threshold=0):
    img[np.abs(img) > threshold] = 255
    return img

def lowpassFilter(img, kernelSize, sigma=1.0, mu=0.0):
    x, y = np.meshgrid(np.linspace(-1, 1, kernelSize), np.linspace(-1, 1, kernelSize))
    d = np.sqrt(x*x + y*y)
    kernel = np.exp(-((d - mu)**2 / (2.0 * sigma**2)))

    smoothed = cv2.filter2D(img, -1, kernel)
    return smoothed

In [84]:
i = 0

while True:
    frame = cv2.imread(itoimg[i])
    frame_g = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    frame_g = cv2.GaussianBlur(frame_g, (5,5), 0)

    edges = cv2.Canny(frame_g, 50, 200)
    # edges = cv2.Sobel(frame_g, cv2.CV_64F, dx=1, dy=0, ksize=5)

    _frame = edges
    # _frame = cv2.medianBlur(_frame, 3)
    _frame = lowpassFilter(_frame, 55)
    ret, _frame = cv2.threshold(_frame, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    out = cv2.bitwise_and(frame, frame, mask=_frame)
    kps = detector.detect(frame)
    kps_good = []

    kps_pt = [kp.pt for kp in kps]
    kps_mean_col = [out[np.uint16(kp[1])][np.uint16(kp[0])] for kp in kps_pt]
    for idx in range(len(kps_mean_col)):
        if not np.any(kps_mean_col[idx]):
            continue
        kps_good.append(kps[idx])
    out_kps = cv2.drawKeypoints(out, kps_good, None)
    cv2.putText(out_kps, str(i+1), (0, 50), cv2.FONT_HERSHEY_COMPLEX, 2, (0, 255, 0), 3)

    print(np.mean(_frame))

    i+=1

    # _frame = cv2.resize(_frame, (640, 480), interpolation= cv2.INTER_LINEAR)

    cv2.imshow("frame", out)
    cv2.imshow("detections", out_kps)
    key = cv2.waitKey(50)
    if key == 27:
        cv2.destroyAllWindows()
        break

    # fix for notebook
    if i == n:
        cv2.destroyAllWindows()
        break


107.3955078125
108.751025390625
109.420068359375
112.315380859375
115.990966796875
115.3576171875
110.95654296875
103.367138671875
105.4830078125
92.9255859375
87.270263671875
86.518212890625
87.876220703125
88.717919921875
92.25654296875
101.311865234375
107.341552734375
111.87294921875
115.0953125
116.689892578125
117.277587890625
117.312451171875
116.36533203125
117.39296875
120.082421875
121.5267578125
121.175634765625
119.757861328125
117.3830078125
116.68740234375
112.594287109375
108.224755859375
102.9197265625
96.965576171875
90.47021484375
89.8974609375
90.992333984375
91.256298828125
91.151708984375
93.446875
97.473583984375
101.839794921875
105.36845703125
107.710107421875
108.663037109375
107.872802734375
